In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [4]:
#What is pipeline
#its is a mechnisam which chain together multiple step the onput of one step is the output of next step.
#individua or together (papeline make it easy to apply the same preprocessing to train and split).
#we will do a project with pipeline or without pipeline.

In [31]:
titanic=sns.load_dataset("titanic")
titanic.head(4)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False


In [86]:
titanic["embark_town"].value_counts()

embark_town
Southampton    644
Cherbourg      168
Queenstown      77
Name: count, dtype: int64

In [32]:
titanic.drop(columns=["who","adult_male","alive","alone","deck"],inplace=True)

In [56]:
titanic.drop(columns=["embarked"],inplace=True)

In [72]:
x=titanic[["pclass","sex","age","sibsp","parch","fare","embark_town"]]
y=titanic[["survived"]]

In [58]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [62]:
s1_imputer=SimpleImputer()
s1_embark_town=SimpleImputer(strategy="most_frequent")

In [114]:
#impute for train_data
x_train_age=s1_imputer.fit_transform(x_train[["age"]])
x_train_embar_town=s1_embark_town.fit_transform(x_train[["embark_town"]])
#impute the test data
x_test_age=s1_imputer.fit_transform(x_test[["age"]])
x_test_embar_town=s1_embark_town.fit_transform(x_test[["embark_town"]])

In [137]:
Ohe_sex=OneHotEncoder(sparse_output=False,handle_unknown="ignore",dtype="int32")
Ohe_embark_town=OneHotEncoder(sparse_output=False,handle_unknown="ignore",dtype="int32")
#OnehOTencod the train data
x_train_sex=Ohe_sex.fit_transform(x_train[["sex"]])
x_train_embark_town=Ohe_embark_town.fit_transform(x_train[["embark_town"]])
#OnehOTencod the test data
x_test_sex=Ohe_sex.fit_transform(x_test[["sex"]])
x_test_embark_town=Ohe_embark_town.fit_transform(x_test[["embark_town"]])

In [133]:
x_train_embark_town

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [103]:
x_train_rem=x_train.drop(columns=["sex","age","embark_town"])

,pclass,sibsp,parch,fare
331,1,0,0,28.5000
733,2,0,0,13.0000
382,3,0,0,7.9250
704,3,1,0,7.8542
813,3,4,2,31.2750
...,...,...,...,...
106,3,0,0,7.6500
270,1,0,0,31.0000
860,3,2,0,14.1083
435,1,1,2,120.0000


In [161]:
x_test_rem=x_test.drop(columns=["sex","age","embark_town"])

In [140]:
x_train_transformed=np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_embar_town),axis=1)
x_test_tranformed=np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_embar_town),axis=1)

In [148]:
print(x_train_transformed.shape)
print(x_test_tranformed.shape)

(712, 8)
(179, 8)


In [154]:
#Now i will do the dame thing with the sklearn.pipleline.
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [162]:
tc=sns.load_dataset("titanic")
tc.head(4)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False


In [155]:
#the plain is age and embarked have misiing data fill with impute by using colum transfrmer then next step onhehot encode
#then scaling after that we do the feature selection  at we choose best 5 columns.then train model using disicion tree

In [163]:
tc.drop(columns=["class","who","adult_male","alive","alone","deck","embark_town"],inplace=True)

In [170]:
x=tc[["pclass","sex","age","sibsp","parch","fare","embarked",]]
y=tc[["survived"]]

In [171]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [183]:
print(x_train.sample(2))
print(y_train.sample(2))
print("Embaraked Null value are:",tc["embarked"].isnull().sum())

     pclass     sex   age  sibsp  parch  fare embarked
345       2  female  24.0      0      0  13.0        S
160       3    male  44.0      0      1  16.1        S
     survived
316         1
267         1
Embaraked Null value are: 2


In [188]:
#Imputing_transfomration:
trf1=ColumnTransformer([
    ("impute_age",SimpleImputer(),[2]),
    ("embarak_impute",SimpleImputer(strategy="most_frequent"),[6])
],remainder="passthrough")

In [189]:
#One hot Encoding
trf2=ColumnTransformer([
    ("Ohe_sex_embarked",OneHotEncoder(sparse_output=False,handle_unknown="ignore"),[1,6])
],remainder="passthrough")

In [190]:
#Scaling
trf3=ColumnTransformer([
    ("scale",MinMaxScaler(),slice(0,10)) #use use min max becuae we do feature extraction
])

In [191]:
#feature Seletion
trf4=SelectKBest(score_func=chi2,k=8)

In [193]:
#just make the object of DisiconTreeClassifier
trf5=DecisionTreeClassifier()

In [194]:
#Now Create Pipeline
pipe=Pipeline([
    ("trf1",trf1),
    ("trf2",trf2),
    ("trf3",trf3),
    ("trf4",trf4),
    ("trf5",trf5)
])

In [195]:
#pipeline vs make_pipeline
pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [197]:
#train the data
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('embarak_impute',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001EA783D0360>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [219]:
#explore pipeline
pipe.named_steps["columntransformer-1"].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [221]:
y_pred=pipe.predict(x_test)

In [226]:
#check accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)*100 #its not good because we do featur extraction and model train on least columns

62.56983240223464

In [236]:
#Cross validation using pipeline
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train,cv=5,scoring="accuracy").mean()*100

63.91214419383433

In [237]:
#explore the pipeline
import pickle
pickle.dump(pipe,open('pipe.pkl',"wb"))